In [13]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import numpy as np
import tensorboard
import optuna

In [14]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward, contact_cost_weight, healthy_z_range, contact_force_range):
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium con i parametri specificati.
    """
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight, 
                    healthy_reward=healthy_reward, 
                    contact_cost_weight = contact_cost_weight,
                    healthy_z_range=healthy_z_range,
                    contact_force_range=contact_force_range)
                   # render_mode='none')

In [15]:
# Hyperparameter tuning con Optuna

def objective(trial):
    # Parametri dell'environment
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.05, 0.2)           # Default circa 0.1; esploriamo da 0.05 a 0.2
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.5, 1.5)     # Default tipico è 1; esploriamo da 0.5 a 1.5
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.1, 1.0)               # Default tipico 0.5; esploriamo da 0.1 a 1.0
    healthy_reward = trial.suggest_float('healthy_reward', 0.5, 1.5)                   # Default tipico 1; esploriamo da 0.5 a 1.5
    
    # Parametri aggiuntivi per Ant-v5
    contact_cost_weight = trial.suggest_float('contact_cost_weight', 1e-4, 1e-3)  # Es. range intorno a 5e-4 come default
    healthy_z_lower = trial.suggest_float('healthy_z_lower', 0.1, 0.3)             # Per definire l'intervallo di altezze "sane"
    healthy_z_upper = trial.suggest_float('healthy_z_upper', 0.8, 1.2)
    contact_force_min = trial.suggest_float('contact_force_min', -1.0, -0.5)         # Modificabile se usi forze di contatto
    contact_force_max = trial.suggest_float('contact_force_max', 0.5, 1.0)  

    # Crea l'ambiente passando tutti i parametri
    env = make_env(
        reset_noise_scale,
        forward_reward_weight,
        ctrl_cost_weight,
        healthy_reward,
        contact_cost_weight=contact_cost_weight,
        healthy_z_range=(healthy_z_lower, healthy_z_upper),
        contact_force_range=(contact_force_min, contact_force_max)
    )
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

    # Iperparametri per il modello PPO
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    n_steps = trial.suggest_int('n_steps', 2048, 8192, step=2048)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])  
    # Per ambienti complessi come Ant, molti esperimenti usano gamma intorno a 0.99-0.995
    gamma = trial.suggest_float('gamma', 0.99, 0.999)
    gae_lambda = trial.suggest_float('gae_lambda', 0.8, 1.0)
    clip_range = trial.suggest_float('clip_range', 0.1, 0.3) 
    ent_coef = trial.suggest_float('ent_coef', 0.0, 0.1)

    # Crea ed allena il modello PPO
    model = PPO("MlpPolicy", env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                gamma=gamma,
                gae_lambda=gae_lambda,
                clip_range=clip_range,
                ent_coef=ent_coef,
                verbose=0)
    model.learn(total_timesteps=200000)
    '''
    # Valuta il modello su 200 episodi (200 è ottimale)
    episodes = 150
    episode_rewards = []
    for episode in range(episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)'''
    episodi= 150
    # Evaluate the model
    mean_reward, _ = evaluate_policy(model, env, episodi, deterministic=True)

    # Calcola reward media 
    #mean_reward = np.mean(episode_rewards)

    print(f'Mean is: {mean_reward}')

    return mean_reward

# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)

[I 2025-02-12 20:07:02,535] A new study created in memory with name: no-name-be910a3d-8ef7-451f-99d9-89d1fed12d45
/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_63463/2145312873.py:31: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
[W 2025-02-12 20:08:29,870] Trial 0 failed with parameters: {'reset_noise_scale': 0.1350487877015317, 'forward_reward_weight': 1.146694973637576, 'ctrl_cost_weight': 0.9829063390143424, 'healthy_reward': 0.5668862522269956, 'contact_cost_weight': 0.0007679249726954685, 'healthy_z_lower': 0.12135524500956488, 'healthy_z_upper': 0.9061735934555887, 'contact_force_min': -0.7303904488287072, 'contact_force_max': 0.9190780228763508, 'learning_rate': 0.00021378756879846382, 'n_steps': 4096, 'batch_size': 256, 'gamma': 0.996577

TypeError: evaluate_policy() got an unexpected keyword argument 'episodes'